# scraping FTSE30 data

## 一、构建类获取股票交易数据
数据API: https://www.alphavantage.co/documentation/  API KEY:34A4VP32WLWXG9WG

由于这个API的python版document写的不详细，因此我们选择自己解析获取到的数据并且使用pandas处理
### 从网上读取数据，选择想要的股票数据，将json数据格式解析成pandas Frame 用于后续计算

In [1]:
import urllib3
import json
import pandas as pd
class Stock(object):
    def __init__(self):
        self.api_address = "https://www.alphavantage.co/"
        self.key = "34A4VP32WLWXG9WG"
        #       首先根据urllib3的文档，建立http连接池
        self.http = urllib3.PoolManager()
        
#   以日为单位，获取交易信息记录，输入 股票symbols(列表)， 返回近20年的股票记录
    def stocks_daily_query(self, symbols):
        
#       init 总的dataFrame作为返回值，假定行表示股票名，列是时间，数据是close price
        stock_frame = pd.DataFrame()
        

#       定义function为TIME_SERIES_DAILY 表示以日为单位查询
        function = "TIME_SERIES_DAILY"
    
    
#       循环开始：遍历所有的symbols,  股票名为symbol
        for symbol in symbols:
#           动态构建获取数据的地址 web_address 
#           地址格式如同 https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=demo
#             web_address = "%squery?function=%s&symbol=%s&apikey=%s"    % (self.api_address, function, symbol,self.key)
            web_address = "%squery?function=%s&symbol=%s&outputsize=full&apikey=%s"    % (self.api_address, function, symbol,self.key)
#           使用http.request请求页面
            r = self.http.request('GET', web_address)
#           请求得到的r.data是byte格式，先转换成str 再使用 json 格式保存
            j = json.loads(r.data.decode('utf-8'))
            if len(j)>=2:
#                 print(j)
    #           这个字段中只有time series (Daily)的信息有用，将其中的信息取出,并将格式从dict转换成str
                d = json.dumps(j["Time Series (Daily)"])

    #           构建 pandas dataFrame来存储数据
                df = pd.read_json(d, orient='index')
    #           需要close价格，取出该列。并改名
                close_price = df.loc[:,['4. close']]
                close_price.columns = ["%s_price" % symbol]
    #           将该列加入到总的stock_frame中
                stock_frame = pd.concat([stock_frame,close_price],axis=1)
    #           需要volume总量取出该列。并改名
                volume = df.loc[:,['5. volume']]
                volume.columns = ["%s_volume" % symbol]
    #           将该列加入到总的stock_frame中
                stock_frame = pd.concat([stock_frame,volume],axis=1)
    
        
        return stock_frame

In [2]:
p = Stock()
# FTSE100 = [    "AAL.L","ABF.L","ADM.L","ADN.L","AGK.L","UU.L","VED.L","VOD.L",
#                "AMEC.L","ANTO.L","ARM.L","ASHM.L","AV.L",
#                "AZN.L","BA.L","BARC.L","BATS.L","BG.L","BLND.L",
#                "BLT.L","BNZL.L","BP.L","BRBY.L","BSY.L","BT-A.L",
#                "CCL.L","CNA.L","CPG.L","CPI.L","CRDA.L","CRH.L","CSCG.L",
#                "DGE.L","EMG.L","ENRC.L","EVR.L","EXPN.L","FRES.L","GFS.L",
#                "GKN.L","GLEN.L","GSK.L","HL.L","HMSO.L","HSBA.L","IAG.L","IAP.L",
#                "IHG.L","IMI.L","IMT.L","IPR.L","ITRK.L","ITV.L","JMAT.L","KAZ.L",
#                "KGF.L","LAND.L","LGEN.L","LLOY.L","MGGT.L","MKS.L","MRW.L","NG.L",
#                "NXT.L","OML.L","PFC.L","POLY.L","PRU.L","PSON.L","RB.L","RBS.L",
#                "RDSB.L","REL.L","REX.L","RIO.L","RR.L","RRS.L","RSA.L","RSL.L","SAB.L",
#                "SBRY.L","SDR.L","SDRC.L","SGE.L","SHP.L","SL.L","SMIN.L","SN.L",
#                "SRP.L","SSE.L","STAN.L","SVT.L","TATE.L","TLW.L","TSCO.L","ULVR.L",
#                "WEIR.L","WOS.L","WPP.L","WTB.L","XTA.L"]

FTSE30 = ["AAL.L","ABF.L","BA.L","BATS.L","BT.L","BRBY.L","CPG.L","CCL.L","CNA.L",
          "DGE.L","EXPN.L","GKN.L","GSK.L","IAG.L","ITV.L","LAND.L",
         "LLOY.L","EMG.L","MKS.L","RB.L","RBS.L","RSA.L","SMIN.L","RIO.L","RR.L","SHP.L",
         "TATE.L","TSCO.L","VOD.L","WPP.L","WTB.L","XTA.L"]
stock_frame = p.stocks_daily_query(FTSE30)
stock_frame = stock_frame.loc['2014-12-01':'2017-12-01',:]

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packag

In [5]:
import numpy as np
# stock_frame
# stock_frame.index[np.where(np.isnan(stock_frame))[0]]
stock_frame.to_csv("FTSE30.csv")